# Deploy App to Snowflake

In [1]:
from snowflake.snowpark import Session
from string import Template
import json

### Connect to Snowflake

You can create a session however you like. Here are two possible options. 

OPTION 1 - Using builder.getOrCreate() to access an existing toml file
- https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-connect#connecting-using-the-connections-toml-file

In [2]:
session = Session.builder.getOrCreate()

OPTION 2 - Using connection params inside builder.configs().create() 

In [3]:
# connection_params = dict(
#     user="",
#     role="",
#     password="",
#     account="",
# )

# session = Session.builder.configs(connection_params).create()

### Establish metadata

NOTE: In the metadata dict below, you can change any of the key:value pairs EXCEPT main_file="automl_app.py".

In [4]:
metadata = dict(
    database_name="ML_SIDEKICK",
    schema_name="ST_APPS",
    stage_name="APP_STG",
    app_name="ML_SIDEKICK",
    main_file="automl_app.py",  # DO NOT CHANGE
    query_warehouse="COMPUTE_WH",  # CHANGE TO AN EXISTING WAREHOUSE
)
with open("deployment_structure.json", "r") as config:
    upload_metadata = json.loads(config.read())

### Templates

In [12]:
create_db = Template("CREATE DATABASE IF NOT EXISTS $db")
create_schema = Template("CREATE SCHEMA IF NOT EXISTS $db.$schema")
create_stage = Template(
    """
CREATE STAGE IF NOT EXISTS $db.$schema.$stage
DIRECTORY=(ENABLE=TRUE);
"""
)
create_streamlit = Template(
    """CREATE STREAMLIT IF NOT EXISTS $db.$schema.$app_name
  ROOT_LOCATION = '@$db.$schema.$stage'
  MAIN_FILE = '$main_file'
  QUERY_WAREHOUSE = $wh
  COMMENT = '{"origin":"sf_sit", "name":"ml_sidekick", "version":{"major":1, "minor":0}, "attributes":{"component":"sis_app"}}'
  """
)

##### Populate Templates

In [13]:
db_query = create_db.substitute(db=metadata.get("database_name"))

schema_qry = create_schema.substitute(
    db=metadata.get("database_name"), schema=metadata.get("schema_name")
)

stage_qry = create_stage.substitute(
    db=metadata.get("database_name"),
    schema=metadata.get("schema_name"),
    stage=metadata.get("stage_name"),
)

app_create_qry = create_streamlit.substitute(
    app_name=metadata.get("app_name"),
    db=metadata.get("database_name"),
    schema=metadata.get("schema_name"),
    stage=metadata.get("stage_name"),
    main_file=metadata.get("main_file"),
    wh=metadata.get("query_warehouse"),
)

In [7]:
session.sql(db_query).collect()

[Row(status='AUTO_ML already exists, statement succeeded.')]

In [8]:
session.sql(schema_qry).collect()

[Row(status='ST_APPS already exists, statement succeeded.')]

In [9]:
session.sql(stage_qry).collect()

[Row(status='APP_STG already exists, statement succeeded.')]

### Upload project files

In [10]:
db = metadata.get("database_name")
schema = metadata.get("schema_name")
stage = metadata.get("stage_name")
for i in upload_metadata.get("files"):
    for file in i.get("files"):
        path = "" if i.get("parent") == "root" else i.get("parent")
        session.file.put(
            local_file_name=file,
            stage_location=f"@{db}.{schema}.{stage}/{path}",
            auto_compress=False,
            overwrite=True,
        )

# Finally, Create the app

In [14]:
session.sql(app_create_qry).collect()

[Row(status='Streamlit STREAMLIT_AUTO_ML successfully created.')]